In [1]:
import os
import json
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
import pandas as pd
import numpy as np
import xarray as xr
import rioxarray as rxr
import geoviews as gv
import geoviews.feature as gf
import hvplot.pandas
import s3fs
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from multiprocessing import Pool, cpu_count
import dask
from dask.distributed import Client, progress, LocalCluster
import dask.array as da
import geopandas as gpd

In [2]:
# THIS IS FROM EMMA's TUTORIAL https://github.com/e-marshall/itslivetools/blob/main/itslivetools/access.py
def find_granule_by_point(input_point, label='single_point'):
    '''returns url for the granule (zarr datacube) containing a specified point. point must be passed in epsg:4326
    '''
    catalog = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json')

    #make shapely point of input point
    p = gpd.GeoSeries([Point(input_point[0], input_point[1])],crs='EPSG:4326')
    #make gdf of point
    gdf = gdf = gpd.GeoDataFrame({'label': f'{label}', 
                                  'geometry':p})

    #find row of granule 
    granule = catalog.sjoin(gdf, how='inner')

    url = granule['zarr_url'].values[0]
    return url
    
def read_in_s3(http_url, chunks='auto'):
    ''' does some string formatting from zarr url and returns xarray dataset
    '''

    s3_url = http_url.replace('http','s3')
    s3_url = s3_url.replace('.s3.amazonaws.com','')

    datacube = xr.open_dataset(s3_url, engine='zarr',
                               storage_options={'anon':True},
                               chunks=chunks,
                              # lock=False
                              )

    # datacube = rxr.open_rasterio(s3_url, engine='zarr',
    #                            storage_options={'anon':True},
    #                            chunks=chunks)
    
    return datacube


In [3]:
catalog = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json') # ITS_LIVE data catalog where all the paths are defined
path = './geoms/2020_12_11_term_polygon.shp'
kuj_poly = gpd.read_file(path)
kuj_centroid = kuj_poly.centroid
kuj_centroid = gpd.GeoDataFrame(geometry=kuj_centroid)#,crs=4326)
granule = catalog.sjoin(kuj_centroid, how='inner')
url = granule['zarr_url'].values[0]

kuj_vels = read_in_s3(url)
kuj_vels = kuj_vels.chunk({"mid_date": len(kuj_vels.mid_date)}) # setting the mid_date chunksize to the time dimensions helps A LOT
kuj_vels = kuj_vels.sortby('mid_date')
kuj_vels.rio.write_crs('EPSG:3413',inplace=True)

/tmp/ipykernel_8366/2972028994.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  kuj_centroid = kuj_poly.centroid


<xarray.Dataset>
Dimensions:                     (mid_date: 49475, y: 834, x: 833)
Coordinates:
    mapping                     int64 0
  * mid_date                    (mid_date) datetime64[ns] 1984-10-21T14:23:16...
  * x                           (x) float64 -1.999e+05 -1.997e+05 ... -1e+05
  * y                           (y) float64 -2.1e+06 -2.1e+06 ... -2.2e+06
Data variables: (12/59)
    M11                         (mid_date, y, x) float32 dask.array<chunksize=(49475, 20, 20), meta=np.ndarray>
    M11_dr_to_vr_factor         (mid_date) float32 dask.array<chunksize=(49475,), meta=np.ndarray>
    M12                         (mid_date, y, x) float32 dask.array<chunksize=(49475, 20, 20), meta=np.ndarray>
    M12_dr_to_vr_factor         (mid_date) float32 dask.array<chunksize=(49475,), meta=np.ndarray>
    acquisition_date_img1       (mid_date) datetime64[ns] dask.array<chunksize=(49475,), meta=np.ndarray>
    acquisition_date_img2       (mid_date) datetime64[ns] dask.array<chunksize=(49475,), meta=np.ndarray>
    ...                          ...
    vy_error_modeled            (mid_date) float32 dask.array<chunksize=(49475,), meta=np.ndarray>
    vy_error_slow               (mid_date) float32 dask.array<chunksize=(49475,), meta=np.ndarray>
    vy_error_stationary         (mid_date) float32 dask.array<chunksize=(49475,), meta=np.ndarray>
    vy_stable_shift             (mid_date) float32 dask.array<chunksize=(49475,), meta=np.ndarray>
    vy_stable_shift_slow        (mid_date) float32 dask.array<chunksize=(49475,), meta=np.ndarray>
    vy_stable_shift_stationary  (mid_date) float32 dask.array<chunksize=(49475,), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.8
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               26-Sep-2023 03:06:33
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v2/N70W040/ITS_L...
    skipped_granules:           s3://its-live-data/datacubes/v2/N70W040/ITS_L...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...

### Here I created a merged polygon of all the terminus polygons of Kuj that I had. Maybe there are more, but having a sinlge file (in my experience preferably geopackage for larger data) makes everything much simpler 

In [4]:
term_polys_path = '/media/laserglaciers/upernavik/its_live_tutorial/merged_geoms/kuj_merged.gpkg'
term_polys = gpd.read_file(term_polys_path)

/home/laserglaciers/anaconda3/envs/its_live_tutorial/lib/python3.10/site-packages/geopandas/io/file.py:399: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  as_dt = pd.to_datetime(df[k], errors="ignore")


### Here I rely on numpy to filter out the matching dates between the its_live data cube and terminus polygons
### the np.where(np.isin(kj_dates,term_dates_pd)) statement is very efficient for this

In [5]:
kj_dates = kuj_vels.mid_date.dt.date.data
term_dates_pd = term_polys['date'].dt.date.values
matched_idx = np.where(np.isin(kj_dates,term_dates_pd)) # returns the indices of the matched dates so we can just use an isel (integer select) on the data cube
kuj_vels = kuj_vels.isel(mid_date=matched_idx[0])

### This is the main function that clips and counts the number of pixels for the mask area %

In [6]:
def rio_clip(dataset,gdf,date_list,crs):
    velocity = []
    area = []
    for date in date_list:
        geom_values = gdf[gdf['date'] == pd.to_datetime(date.date())].geometry.values
        dataset_sample_clip = dataset.v.sel(mid_date=date).rio.clip(geom_values,crs,drop=True,invert=False)
        mean_arr = dataset_sample_clip.mean(dim=["x", "y"],skipna=True)
        area_count = dataset_sample_clip.count(dim=["x", "y"]) #,skipna=True) This should not count NaNs
        
        velocity.append(mean_arr)
        area.append(area_count)
        
    return area, velocity

In [7]:
date_list = pd.to_datetime(kuj_vels.mid_date) #creates a list which we will split up to divide the parallelized work 

### Splitting up the data can be tricky and depends on the number of cores and memory you plan to run it on. This worked for me to get through SOME of the data

In [8]:
split_size = 300
date_lists = [date_list[x:x+split_size] for x in range(0, len(date_list), split_size)]

In [9]:
len(date_lists)

33

### args stores the arguments to run in the multiproccessing pool. Noticed how I only did the first 4 date_lists entries. I was having trouble with running out of memory using the full list

In [10]:
args = []
crs = term_polys.crs
for date_list in date_lists[:4]:
    args.append((kuj_vels,term_polys,date_list,crs))

### `result` stores the output of the `rio_clip` function as lists of dask arrays ready to be computed

In [11]:
with Pool(processes=cpu_count()-8) as pool:
    result = pool.starmap(rio_clip, args)
    

### Now we have to reorginize the data. `results` is organized like this `results[i][0]` are the lists where the pixel count dask arrays are stored. `results[i][1]` are where the velocity averages are stored. The cell below uses `xr.concat` to combine all the pixel counts and the velocity means into their own xarray data arrays

In [12]:
computed_area = []
for i in range(0,4):
    computed = dask.compute(*result[i][0])
    computed_area.append(computed)
    
computed_mean_vel = []
for i in range(0,4):
    computed_vel = dask.compute(*result[i][1])
    computed_mean_vel.append(computed_vel)



### Now we need to reduce from a list of tuples to just a single list for each data variable

In [14]:
computed_area_explode = [item for t in computed_area for item in t]
computed_vel_explode = [item for t in computed_mean_vel for item in t]

### Now we use `xarray.concat` to create our data arrays

In [15]:
area_count = xr.concat(computed_area_explode,"mid_date").rename('area_count') # this creates one big data array
v_mean = xr.concat(computed_vel_explode,"mid_date").rename('v_mean') # this creates one big data array

### To create a single dataset we use `xarray merge`

In [16]:
area_vel_merge = xr.merge([area_count,v_mean])
area_vel_merge

<xarray.Dataset>
Dimensions:     (mid_date: 1310)
Coordinates:
  * mid_date    (mid_date) datetime64[ns] 1985-03-29T14:30:57.925833024 ... 2...
    mapping     int64 0
Data variables:
    area_count  (mid_date) int64 0 437 365 0 165 28 0 0 ... 150 47 0 0 0 778 0
    v_mean      (mid_date) float32 nan 3.126e+03 3.065e+03 ... nan 2.585e+03 nan

### The resulting dataset has both the area_count and v_mean stored together and tied to the same `mid_date` coordinate. 

### I have not applied the terminus polygon area to this to create the % of pixel area covered. Though I am assuming it will be faster with the single geopackage and can be matched by the date columns 